In [1]:
from dm import Country, utils

In [7]:
df = utils.get_countries()

df

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2019,USA,2019
2,USA_ESRI_2020,USA,2020


In [8]:


df[df['cntry']]

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2019,USA,2019
2,USA_ESRI_2020,USA,2020


In [4]:
usa

<dm.Country - USA (local)>